In [4]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary

torch.backends.cudnn.benchmark = True
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
torch.from_numpy(2)

RuntimeError: Numpy is not available

In [18]:
import networkx as nx
import torch_geometric.nn as gnn
import torch_geometric.data as gd
import torch.nn.functional as F

class SignalGCN(nn.Module):
    def __init__(self):
        super(SignalGCN, self).__init__()
        self.conv1 = gnn.GCNConv(1, 256)
        self.pool1 = gnn.global_mean_pool()
        self.conv2 = gnn.GCNConv(128,64)
        self.pool2 = gnn.global_mean_pool()
        #self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(32, 16)
        self.d1 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(16, 6)

    def forward(self, x):
        # Create graph from 1D signal
        data = encode_signal(x)

        # Pass through GCN
        print(data.x.shape)
        x = F.relu(self.conv1(data.x, data.edge_index))
        print("First conv", x.shape)
        x = self.pool1(x, batch)
        print("First pool", x.shape)
        #x = x.view(-1, 32, 128)
        #print(x.shape)
        x = F.relu(self.conv2(x, data.edge_index))
        x = self.pool2(x)
        #x = x.view(x.size(0), -1)
        print("final pool", x.shape)
        #x = F.relu(self.fc1(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))
        x = self.d1(x)
        x = self.fc3(x)
        #print(x.shape)
        return x

In [256]:
dataset[0]

{'input': tensor([[-7.7464e-05,  1.3985e-03,  1.7646e-03,  1.9350e-03,  2.0490e-03,
           2.1343e-03,  2.2026e-03,  2.2598e-03,  2.3090e-03,  2.3519e-03,
           2.3897e-03,  2.4231e-03,  2.4528e-03,  2.4791e-03,  2.5026e-03,
           2.5236e-03,  2.5423e-03,  2.5590e-03,  2.5740e-03,  2.5875e-03,
           2.5995e-03,  2.6104e-03,  2.6201e-03,  2.6288e-03,  2.6367e-03,
           2.6438e-03,  2.6502e-03,  2.6559e-03,  2.6611e-03,  2.6657e-03,
           2.6699e-03,  2.6737e-03,  2.6771e-03,  2.6801e-03,  2.6829e-03,
           2.6854e-03,  2.6876e-03,  2.6897e-03,  2.6915e-03,  2.6931e-03,
           2.6946e-03,  2.6960e-03,  2.6972e-03,  2.6983e-03,  2.6992e-03,
           2.7001e-03,  2.7009e-03,  2.7017e-03,  2.7023e-03,  2.7029e-03,
           2.7034e-03,  2.7039e-03,  2.7043e-03,  2.7047e-03,  2.7051e-03,
           2.7054e-03,  2.7057e-03,  2.7059e-03,  2.7062e-03,  2.7064e-03,
           2.7066e-03,  2.7067e-03,  2.7069e-03,  2.7070e-03,  2.7072e-03,
           2.707

In [3]:
def encode_signal(signal):
    # Construct graph from signal
    batch_size = signal.shape[0]
    
    graphs = []
    for i in range(batch_size):
        # For simplicity, just create a fully connected graph
        edge_index = torch.tensor([range(len(signal)-1), range(1, len(signal))]).long()
        x = signal[i, 0].reshape(128, 1)
        data = gd.Data(x=x, edge_index=edge_index)
        graphs.append(data)
    
    batch = torch_geometric.data.batch.Batch.from_data_list(graphs)
    batch.to(DEVICE)
    
    
    return batch

In [4]:
def train(train_loader, net, LR=0.5, epochs=3000, val_loader=None):
    net.to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=LR)
    criterion = nn.MSELoss()
    all_MSE = nn.MSELoss()
    val_losses = []
    print(f"Using: {DEVICE}")
                            
    parameter_loss = []
    losses = []
    processed = 0
    last_loss = 0
    for epoch in range(epochs):
        loss = 0
        
        net.train()
        with tqdm(train_loader, unit="batch") as it:
            if epoch > 0:
                it.set_postfix(lastLoss=last_loss, valLoss=val_losses[-1])
            for idx, data in enumerate(it):
                it.set_description(f"Epoch {epoch+1}/{epochs}")
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                cost = criterion(out, predicted)
                loss += cost.item() #* inp.size(0)
                #print(f"Predicted: {list(predicted)}\nActual: {list(out)}")
        
        if val_loader:
            val_loss = 0
            net.eval()
            for idx, data in enumerate(val_loader):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                cost = criterion(out, predicted)
                val_loss += cost.item() * inp.size(0)
            val_loss /= len(val_loader)  
            val_losses.append(val_loss)
        
        losses.append(loss)
        last_loss = loss/len(it)
    print("Parameters: Skin YM, Adipose YM, Skin PR, Adipose PR, Skin Perm, Adipose Perm")
    print(f"Sampled Ranges: 10e3 - 50e3, 1e3 - 25e3, 0.48 - 0.499, 0.48 - 0.499, 10e - 12-10e10, 10e-12 - 10e10") 
    #print(f"Average parameter loss: {np.mean(np.reshape(np.array(parameter_loss), (-1, 6)), axis=0)}")        
    print(f"Average overall loss: {np.sum(losses)/epochs}")
    return losses, parameter_loss, val_losses

def test(test_loader, net):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()
    crit = nn.L1Loss()
    

    with torch.no_grad():
            loss = 0
            with tqdm(test_loader, unit=" batch") as it:
                for idx, data in enumerate(it):
                    inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)
                    
                    predicted = net(inp)
                    cost = criterion(out, predicted)
                    loss += cost.item() * inp.size(0)
                    
                    #print(f"\n\n\nBatch: {idx}")
                   # print(f"loss: {l_t}")
                    #for i, target in enumerate(out):
                   #     errs = []
                   #     print(f"Targer: {target}, \npredicted: {predicted[i]}\n\n")
                   #     for j in range(len(predicted)):
                   #         errs.append(abs(predicted[i]-target[i])**2)
                   #     print(f"MSE: {np.mean(errs[0])}")
            
            print(f"Average Loss: {loss/len(test_loader)}") 
            
torch.backends.cudnn.benchmark = True

In [5]:
def train(model, optimizer, loss_fn, train_loader, device):
    model.train()
    train_loss = 0.0
    for data in train_loader:
        graph, target = data
        graph, target = graph.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(graph)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * graph.num_graphs
    return train_loss / len(train_loader.dataset)

def test(model, loss_fn, test_loader, device):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            graph, target = data
            graph, target = graph.to(device), target.to(device)
            output = model(graph)
            loss = loss_fn(output, target)
            test_loss += loss.item() * graph.num_graphs
    return test_loss / len(test_loader.dataset)


In [6]:
with open("filtered.pkl", "rb") as f:
    runs = pickle.load(f)

In [7]:
from torch.utils.data import Dataset
from torch import tensor

# Folder name will correspond to index of sample
class Data(Dataset):
    def __init__(self, signalFolder, sampleFile, runs, steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)

        self.min = [min(np.array(samples[runs])[:, i]) for i in range(6)]
        self.max = [max(np.array(samples[runs])[:, i]) for i in range(6)]
        
        for run in runs: #os.listdir(f"{signalFolder}/"):  
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                if a['x'].max() != 7.0:
                    fail = True
                    break
                #a = pd.concat([pd.DataFrame([[0,0.1]], columns=a.columns), a], ignore_index=True)
                

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                #fix, ax = plt.subplots(1, 2)
                #ax[0].plot(xNew, interped)
                #a.plot(ax=ax[1], x='x', y='y')
                #if run == 3:
                   # break
                
                
                #if len(a) < 702:
                #    print(f"{signalFolder}/{run}/{file}: {len(a)}")
               #     fail = True
                 #   break
                
               # while len(a) > 702:
                #    a = a.drop(index=np.random.randint(0, len(a)-1)).reset_index(drop=True)
                
                #print(a)
                
                inp.append(list(interped))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")
                #print(inp[0])
                #raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        self.output = tensor(self.output)
        self.input = tensor(self.input)
        #for dim in range(6):
       #     self.output[:, dim] = (self.output[:, dim] - self.min[dim])/(self.max[dim] - self.min[dim])
        #print(self.output)
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample

In [8]:
from torch_geometric.data import Batch

def custom_collate(data_list):
    # Stack the `Data` objects into a `Batch` object
    #data_list = [data_list[i] for i in range(len(data_list))]
    print(data_list[0])
    batch = Batch.from_data_list(data_list[0])
    
    batch = {"input": batch, "output": data_list[1]}
    return batch

In [9]:
dataset = Data("D:/SamplingResults", "C:/Users/rjsou/Documents/MastersDiss/Sampling/Samples.pkl", runs)

In [10]:
from torch.utils.data import DataLoader
train_n = int(0.8 * len(dataset))
test_n = len(dataset) - train_n
train_set, test_set = torch.utils.data.random_split(dataset, [train_n, test_n])
train_loader, test_loader = DataLoader(train_set, batch_size=2, shuffle=True, pin_memory=True), DataLoader(test_set, batch_size=2, shuffle=True, pin_memory=True)

In [19]:
net = SignalGCN()

AttributeError: module 'torch_geometric.nn' has no attribute 'GlobalMeanPool'

In [ ]:
train(train_loader, net, val_loader = test_loader)

In [2]:
def train(train_loader, net, LR=0.1, epochs=2000, val_loader=None):
    net.to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=LR)
    criterion = nn.L1Loss()
    all_MSE = nn.L1Loss()
    val_losses = []
    print(f"Using: {DEVICE}")
                            
    parameter_loss = []
    losses = []
    processed = 0
    last_loss = 0
    for epoch in range(epochs):
        loss = 0
        net.train()
        with tqdm(train_loader, unit="batch") as it:
            if epoch > 0:
                it.set_postfix(lastLoss=last_loss, valLoss=val_losses[-1])
            for idx, data in enumerate(it):
                it.set_description(f"Epoch {epoch+1}/{epochs}")
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)
                
                optimizer.zero_grad()
                predicted = net(inp)

                cost = criterion(out, predicted)
                loss += cost.item()
                cost.backward()
                optimizer.step()
                
                for i in range(len(predicted)):
                    current_MSE = []
                    for j in range(6):
                        current_MSE.append(all_MSE(out[i][j], predicted[i][j]).item())
                    parameter_loss.append(current_MSE)
                    processed += 1
        
        if val_loader:
            val_loss = 0
            net.eval()
            for idx, data in enumerate(val_loader):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                cost = criterion(out, predicted)
                val_loss += cost.item()
            val_loss /= len(val_loader)  
            val_losses.append(val_loss)
        
        loss /= len(it)
        losses.append(loss)
        last_loss = loss
    print("Parameters: Skin YM, Adipose YM, Skin PR, Adipose PR, Skin Perm, Adipose Perm")
    print(f"Sampled Ranges: 10e3 - 50e3, 1e3 - 25e3, 0.48 - 0.499, 0.48 - 0.499, 10e - 12-10e10, 10e-12 - 10e10") 
    print(f"Average parameter loss: {np.mean(np.reshape(np.array(parameter_loss), (-1, 6)), axis=0)}")        
    print(f"Average overall loss: {np.sum(losses)/processed}")
    return losses, parameter_loss, val_losses

def test(test_loader, net):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()
    crit = nn.L1Loss()
    differences = []
    outs = []

    with torch.no_grad():
            loss = 0
            with tqdm(test_loader, unit=" batch") as it:
                for idx, data in enumerate(it):
                    inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)
                    
                    predicted = net(inp)
                    cost = criterion(out, predicted)
                    l_t = cost.item()
                    loss += l_t
                    for i in range(len(predicted)):
                        p = predicted[i].cpu().numpy().reshape(1, -1)
                        o = out[i].cpu().numpy().reshape(1, -1)
                        
                        p = SCALER.inverse_transform(p)[0]
                        o = SCALER.inverse_transform(o)[0]

                        
                        differences.append((np.abs(p-o)/o)*100)
                        #print("D", differences[-1])
                        #print(F"Predicted: {SCALER.inverse_transform(p)}")
                        #print(F"Real: {SCALER.inverse_transform(o)}")
                        #print(f"Difference: {abs(p - o)}\n\n")
                        #processed += 1
                    
                    #print(f"\n\n\nBatch: {idx}")
                   # print(f"loss: {l_t}")
            #print(differences[0])
            #print(np.array(differences).shape)
            #print(f"Average Loss: {loss/len(test_loader)}")
            #print(f"Average parameter loss: {np.mean(np.array(differences), axis=0)}") # Dataset
    return np.mean(np.array(differences), axis=0)

In [5]:
# Folder name will correspond to index of sample

from sklearn.preprocessing import MinMaxScaler
SCALER = MinMaxScaler()
class Data(Dataset):
    def __init__(self, signalFolder, sampleFile, runs=range(65535), steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        for run in tqdm(runs): #os.listdir(f"{signalFolder}/"):  
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                if a['x'].max() != 7.0:
                    fail = True
                    break
                #a = pd.concat([pd.DataFrame([[0,0.1]], columns=a.columns), a], ignore_index=True)
                

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                #fix, ax = plt.subplots(1, 2)
                #ax[0].plot(xNew, interped)
                #a.plot(ax=ax[1], x='x', y='y')
                #if run == 3:
                   # break
                
                
                #if len(a) < 702:
                #    print(f"{signalFolder}/{run}/{file}: {len(a)}")
               #     fail = True
                 #   break
                
               # while len(a) > 702:
                #    a = a.drop(index=np.random.randint(0, len(a)-1)).reset_index(drop=True)
                
                #print(a)
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")
                #print(inp[0])
                #raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        SCALER.fit(self.output)
        self.output = SCALER.fit_transform(self.output)
        self.output = tensor(self.output).type(torch.cuda.FloatTensor)
        self.input = tensor(self.input).type(torch.cuda.FloatTensor)
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample

In [7]:
with open("../Data/filtered.pkl", "rb") as f:
    runs = pickle.load(f)

In [8]:
from sklearn.preprocessing import MinMaxScaler
SCALER = MinMaxScaler()

In [9]:
dataset = Data("D:/SamplingResults2", "../Sampling/newSamples.pkl", runs=runs)

100%|██████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:37<00:00, 59.79it/s]
C:\Users\rjsou\AppData\Local\Temp\ipykernel_12804\3404371057.py:73: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  self.input = tensor(self.input).type(torch.cuda.FloatTensor)


In [10]:
train_n = int(0.8 * len(dataset))
test_n = len(dataset) - train_n
train_set, test_set = random_split(dataset, [train_n, test_n])
train_loader, test_loader = DataLoader(train_set, batch_size=32, shuffle=True), DataLoader(test_set, batch_size=32, shuffle=True)

In [13]:
net = Single2D()

In [14]:
tl, pl, vl = train(train_loader, net, val_loader=test_loader, LR=0.0001, epochs=500)

Using: cuda:0


Epoch 1/500:   0%|                                                                           | 0/56 [00:01<?, ?batch/s]

torch.Size([32, 1, 2, 128])


Epoch 1/500:   0%|                                                                           | 0/56 [00:03<?, ?batch/s]


RuntimeError: pad should be at most half of kernel size, but got pad=3 and kernel_size=2

In [12]:
# Up samples to 256 from 128
class Single2D(nn.Module):
    def __init__(self):
        super(Single2D, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, padding=3)
        self.bn1 = nn.BatchNorm2d(128)
        
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(512)
        
        self.conv4 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn5 = nn.BatchNorm2d(128)
        
        
        self.fc1 = nn.Linear(7680, 512)
        self.d1 = nn.Dropout(0.25)
        
        self.fc2 = nn.Linear(512, 256)
        self.d2 = nn.Dropout(0.25)
        
        self.fc3 = nn.Linear(256, 64)
        self.d3 = nn.Dropout(0.25)
        
        self.fc4 = nn.Linear(64, 6)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.unsqueeze(1)
        #x = x.reshape(batch_size, 1, 2, 128)
        print(x.shape)
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
       # x = self.pool4(torch.relu(self.bn4(self.conv4(x))))
        
        x = x.view(batch_size, -1)
        
        x = torch.relu(self.fc1(x))
        x = self.d1(x)
        
        x = torch.relu(self.fc2(x))
        x = self.d2(x)
        
        x = torch.relu(self.fc3(x))
        x = self.d3(x)
        
        
        x = self.fc4(x)
        return x